## Задание 2. SQL
### 2.1 Очень усердные ученики.

### 2.1.1 Условие

Образовательные курсы состоят из различных уроков, каждый из которых состоит из нескольких маленьких заданий. Каждое такое маленькое задание называется "горошиной".

Назовём очень усердным учеником того пользователя, который хотя бы раз за текущий месяц правильно решил 20 горошин.

### 2.1.2 Задача

Дана таблица `default.peas`:

Название атрибута | Тип атрибута | Смысловое значение
------------------|--------------|-------------------
st_id | int | ID ученика
timest | timestamp | Время решения карточки
correct | bool | Правильно ли решена горошина?
subject | text | Дисциплина, в которой находится горошина

Необходимо написать <ins>оптимальный запрос</ins>, который даст информацию о количестве очень усердных студентов.

*NB! Под усердным студентом мы понимаем студента, который правильно решил 20 задач за текущий месяц.*

### 2.2 Оптимизация воронки

### 2.2.1 Условие

Образовательная платформа предлагает пройти студентам курсы по модели trial: студент может решить бесплатно лишь 30 горошин в день. Для неограниченного количества заданий в определенной дисциплине студенту необходимо приобрести полный доступ. Команда провела эксперимент, где был протестирован новый экран оплаты.

2.2.2 Задача

Дана таблицы: `default.peas` (см. выше), `default.studs`:

Название атрибута | Тип атрибута |Смысловое значение
------------------|--------------|-------------------
st_id|	int |	ID ученика
test_grp	|text 	|Метка ученика в данном эксперименте

и `default.final_project_check`:


Название атрибута	|Тип атрибута 	|Смысловое значение
------------------|--------------|-------------------
st_id|	int |	ID ученика
sale_time|	timestamp	|Время покупки
money|	int	|Цена, по которой приобрели данный курс
subject|	text |	Дисциплина, на которую приобрели полный доступ

Необходимо <ins>в одном запросе</ins> выгрузить следующую информацию о группах пользователей:

* ARPU 
* ARPAU 
* CR в покупку 
* СR активного пользователя в покупку 
* CR пользователя из активности по математике (subject = ’math’) в покупку курса по математике

ARPU считается относительно всех пользователей, попавших в группы.

**Активным** считается пользователь, за все время решивший **больше** 10 задач правильно в любых дисциплинах.

**Активным** по математике считается пользователь, за все время решивший **2 или больше задач** правильно по математике.


In [1]:
import pandahouse as ph

In [2]:
connection2 = {'host': 'https://clickhouse.lab.karpov.courses',
                      'database':'default',
                      'user':'student', 
                      'password':'dpo_python_2020'
                     }

In [3]:
query = """
SELECT count(*)
FROM
(SELECT st_id
FROM peas
WHERE correct=1
GROUP BY st_id
HAVING count(*)>=20) as subq
"""

df = ph.read_clickhouse(query, connection=connection2)
df.head()

,count()
0,136


In [4]:
query = '''
SELECT
    test_grp,
    sum(total_revenue) / uniqExact(st_id) as ARPU,
    sumIf(total_revenue, active_stud=1) / uniqExactIf(st_id, active_stud=1) as ARPAU,
    uniqExactIf(st_id, total_revenue > 0) / uniqExact(st_id) as CR_to_buy,
    
    uniqExactIf(st_id, active_stud=1 and total_revenue>0) 
        / uniqExactIf(st_id, active_stud=1) as CR_active_to_buy,
        
    uniqExactIf(st_id, math_active_stud=1 and math_revenue>0) 
        / uniqExactIf(st_id, math_active_stud=1) as CR_math_active_to_buy_math
FROM

    (
    SELECT 
        l.test_grp as test_grp,
        l.st_id as st_id,
        r.active_stud as active_stud,
        r.math_active_stud as math_active_stud,
        r.total_revenue as total_revenue,
        r.math_revenue as math_revenue
    FROM
    
        (
        SELECT
            test_grp,
            st_id
        FROM studs
        ) as l
        
    LEFT JOIN
        (
        SELECT
            l.st_id as st_id,
            l.active_stud as active_stud,
            l.math_active_stud as math_active_stud,
            r.total_revenue as total_revenue,
            r.math_revenue as math_revenue
        FROM
            (
            SELECT
                st_id,
                if (sum(correct) > 10, 1, 0) as active_stud,
                if (sumIf(correct, subject='Math') >= 2, 1, 0) as math_active_stud
            FROM peas
            GROUP BY
                st_id
            ) as l
            
        LEFT JOIN
            (
            SELECT
                st_id,
                sum(money) as total_revenue,
                sumIf(money, subject='Math') as math_revenue
            FROM final_project_check
            GROUP BY st_id
            ) as r
        ON l.st_id = r.st_id
        
        ) as r
    ON l.st_id = r.st_id
    
    )
GROUP BY
    test_grp
'''

df2 = ph.read_clickhouse(query, connection=connection2)
df2.head()

,test_grp,ARPU,ARPAU,CR_to_buy,CR_active_to_buy,CR_math_active_to_buy_math
0,control,4540.983607,10393.700787,0.049180,0.110236,0.061224
1,pilot,11067.796610,29739.583333,0.101695,0.260417,0.095238
